<a href="https://colab.research.google.com/github/liangchow/zindi-amazon-secret-runway/blob/main/image_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports and Setup.

In [ ]:
%%capture
!pip -q install rasterio
!pip -q install torch
!pip -q install torchvision
!pip -q install albumentations
!pip -q install segmentation-models-pytorch
!pip -q install tqdm

In [ ]:
import os
import torch
import rasterio
import albumentations as A
from torch.utils import data
from torch.utils.data import Dataset, DataLoader
import segmentation_models_pytorch as smp
from torchvision import transforms as T
from albumentations.pytorch import ToTensorV2
import torch.nn as nn
import torch.nn.functional as F
import torchsummary
from tqdm import tqdm

# Data manipulation and visualization
import matplotlib.pyplot as plt
from PIL import Image
import imageio
import numpy as np
import cv2
from skimage.measure import label, regionprops
from sklearn.decomposition import PCA


# Set seed for reproducibility
SEED = 42
np.random.seed(SEED)

# Download  data to local compute node

## Mount your Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Compress files, copy over and uncompress

In [ ]:
# Navigate to the shared directory
%cd /content/drive/MyDrive/Zindi-Amazon/training
# Zip the data
!zip -r /content/images.zip images
!zip -r /content/masks.zip masks
!zip -r /content/masks_10m.zip masks_10m
!zip -r /content/masks_20m.zip masks_20m
!zip -r /content/masks_100m.zip masks_100m
!zip -r /content/masks_200m.zip masks_200m
# Unzip the files
!unzip /content/images.zip -d /content
!unzip /content/masks.zip -d /content
!unzip /content/masks_10m.zip -d /content
!unzip /content/masks_20m.zip -d /content
!unzip /content/masks_100m.zip -d /content
!unzip /content/masks_200m.zip -d /content

/content/drive/.shortcut-targets-by-id/14mw0v8Bi-MzhsqSI0K3KO23YrUHttM7P/Zindi-Amazon/training
  adding: images/ (stored 0%)
  adding: images/Sentinel_AllBands_Training_Id_59.tif (deflated 5%)
  adding: images/Sentinel_AllBands_Training_Id_61.tif (deflated 5%)
  adding: images/Sentinel_AllBands_Training_Id_78.tif (deflated 5%)
  adding: images/Sentinel_AllBands_Training_Id_79.tif (deflated 5%)
  adding: images/Sentinel_AllBands_Training_Id_105.tif (deflated 5%)
  adding: images/Sentinel_AllBands_Training_Id_120.tif (deflated 5%)
  adding: images/Sentinel_AllBands_Training_Id_135.tif (deflated 5%)
  adding: images/Sentinel_AllBands_Training_Id_176.tif (deflated 5%)
  adding: images/Sentinel_AllBands_Training_Id_157.tif (deflated 5%)
  adding: images/Sentinel_AllBands_Training_Id_148.tif (deflated 4%)
  adding: images/Sentinel_AllBands_Training_Id_17.tif (deflated 4%)
  adding: images/Sentinel_AllBands_Training_Id_99.tif (deflated 5%)
  adding: images/Sentinel_AllBands_Training_Id_172.ti

# Description of process

Here's a method to identify clusters, fit lines to each one, and extract the endpoints and lengths. We'll start by identifying clusters, then fit lines to each, and finally calculate endpoints and filter based on length-to-width ratios.

**Step-by-Step Method:**
1.   **Identify Clusters:** We can identify clusters of connected pixels in the binary mask using connected component analysis. Each cluster corresponds to a connected component in the binary mask.
2.   **Fit a Line to Each Cluster:** For each identified cluster, fit a line using Principal Component Analysis (PCA), which will help determine the main axis of elongation for the cluster. The endpoints of the fitted line can be computed along this principal component.
3.   **Extract Lengths and Calculate Length/Width Ratios:** Once we have the principal component direction, calculate the length of the line by projecting the points onto the main axis. The width can be computed as the spread of points orthogonal to this main axis, allowing for length-to-width ratio calculations. You can then filter out clusters that don’t meet the criteria.


In [ ]:
def process_clusters(binary_mask, length_width_ratio_cutoff=2.0):
    # Identify clusters in the binary mask
    labeled_mask = label(binary_mask)
    clusters = regionprops(labeled_mask)

    lines = []

    for cluster in clusters:
        # Extract the coordinates of the pixels in the cluster
        coords = cluster.coords

        # Perform PCA to find the main axis
        pca = PCA(n_components=2)
        pca.fit(coords)
        direction = pca.components_[0]
        variance = pca.explained_variance_

        # Compute the length of the cluster along the principal axis
        projected_coords = coords @ direction
        min_proj, max_proj = projected_coords.min(), projected_coords.max()
        length = max_proj - min_proj

        # Calculate width by projecting orthogonal to main axis
        orthogonal_direction = pca.components_[1]
        orthogonal_proj = coords @ orthogonal_direction
        width = orthogonal_proj.max() - orthogonal_proj.min()

        # Compute length-to-width ratio
        length_width_ratio = length / width if width != 0 else np.inf

        # Filter based on length-to-width ratio
        if length_width_ratio >= length_width_ratio_cutoff:
            # Get endpoints of the line along the main axis
            endpoint1 = pca.mean_ + direction * (min_proj - pca.mean_ @ direction)
            endpoint2 = pca.mean_ + direction * (max_proj - pca.mean_ @ direction)
            lines.append({
                "length": length,
                "width": width,
                "ratio": length_width_ratio,
                "endpoints": (endpoint1, endpoint2)
            })

    return lines

In [ ]:
def plot_clusters_and_lines(binary_mask, lines):
    plt.imshow(binary_mask, cmap="gray")
    for line in lines:
        (x1, y1), (x2, y2) = line["endpoints"]
        plt.plot([y1, y2], [x1, x2], 'r-')
    plt.show()


# Main loop

This is where we load the GeoTIFF file prediction, find the clusters,
clean the image and then produce a final prediction. Notice that the final prediction must have a 200 meter buffer.